# **모듈화  함수 사용 예시 및 사용함으로서의 기존 코드 개선 비교**

In [1]:
import pandas as pd

# 데이터 딕셔너리
data = {
    "customer_id": [1001, 1002, 1003, 1001, 1004, 1002, 1005, 1003, 1006, 1001],
    "sales": [5000, 12000, None, 3000, 8000, None, 4500, 7000, 11000, 4000],
    "region": ["North", "South", "East", "North", "West", "South", "East", "East", "West", "North"],
    "product_category": ["Electronics", "Clothing", "Furniture", "Clothing", "Electronics", "Furniture", "Clothing", "Electronics", "Furniture", "Clothing"],
    "signup_date": ["2021-05-20", "2020-08-14", "2021-11-03", "2021-05-20", "2022-01-25", "2020-08-14", "2022-07-07", "2021-11-03", "2022-01-25", "2021-05-20"],
    "date": ["2023-01-15", "2023-02-10", "2023-01-20", "2023-03-05", "2023-01-25", "2023-02-20", "2023-03-10", "2023-02-28", "2023-03-15", "2023-04-01"]
}

# 데이터프레임 생성
df1 = pd.DataFrame(data, index=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

filepath = '/kaggle/input/superstore-dataset-final/Sample - Superstore.csv'
df2 = pd.read_csv(filepath, index_col=0, encoding='latin1')
df1
df2

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932


# **dt_extract**

In [2]:
# pd.to_datetime 재사용 함수화
def dt_extract(
    df: pd.DataFrame, 
    col: str, # 컬럼 타입이 Object 등 다른 타입이어도 상관 X, col의 인자로 직접적으로 컬럼을 가져오는 것이 아니라, col 인자 문자열로 함수 내부에서 특정 컬럼에 접근할 것이기 때문임.
    attr: str,
    new_col: str
) -> pd.DataFrame: 
    """
    데이터프레임의 특정 컬럼을 datetime 타입으로 변환 후 연도, 월, 일 데이터를 추출해 새로운 컬럼을 만드는 함수
    col: datetime으로 변환할 컬럼 / attr: dt에서 추출할 데이터 / new_col: 생성할 컬럼 이름 
    """
    df = df.copy()
    df[col] = pd.to_datetime(df[col])
    df[new_col] = getattr(df[col].dt, attr) # getattr(object, name[, default]) - 객체의 프로퍼티에 동적 접근 가능 / dt의 year, month, day 등 여러 프로퍼티에 필요에 따라 동적으로 접근해야함.

    return df

# 함수 문제점 : datetime으로 변환 후 특정 프로퍼티를 추출하는 케이스가 아닌 경우. 
# year, month, day 모두 추출 필요하거나 / 컬럼 자체를 datetime 변환만 한 후 셀끼리 datetime 단위의 연산 진행할 때

In [3]:
df1
# 예제 코드 모두 pandas_test의 코드이므로 df1 사용

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,customer_id,sales,region,product_category,signup_date,date
1,1001,5000.0,North,Electronics,2021-05-20,2023-01-15
2,1002,12000.0,South,Clothing,2020-08-14,2023-02-10
3,1003,NaN,East,Furniture,2021-11-03,2023-01-20
4,1001,3000.0,North,Clothing,2021-05-20,2023-03-05
5,1004,8000.0,West,Electronics,2022-01-25,2023-01-25
6,1002,NaN,South,Furniture,2020-08-14,2023-02-20
7,1005,4500.0,East,Clothing,2022-07-07,2023-03-10
8,1003,7000.0,East,Electronics,2021-11-03,2023-02-28
9,1006,11000.0,West,Furniture,2022-01-25,2023-03-15
10,1001,4000.0,North,Clothing,2021-05-20,2023-04-01


# Pandas_test 결측치/타입변환 2. 
- "signup_date" 컬럼을 datetime 타입으로 변환해라. 
- 그리고 year, month, day 컬럼을 각각 새로 만들어 추가

In [4]:
# 기존 코드
df1['signup_date'] = pd.to_datetime(df1['signup_date'])
datetime = pd.DataFrame({'year': df1['signup_date'].dt.year, 'month': df1['signup_date'].dt.month, 'day': df1['signup_date'].dt.day})

df1.join(datetime)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,customer_id,sales,region,product_category,signup_date,date,year,month,day
1,1001,5000.0,North,Electronics,2021-05-20,2023-01-15,2021,5,20
2,1002,12000.0,South,Clothing,2020-08-14,2023-02-10,2020,8,14
3,1003,NaN,East,Furniture,2021-11-03,2023-01-20,2021,11,3
4,1001,3000.0,North,Clothing,2021-05-20,2023-03-05,2021,5,20
5,1004,8000.0,West,Electronics,2022-01-25,2023-01-25,2022,1,25
6,1002,NaN,South,Furniture,2020-08-14,2023-02-20,2020,8,14
7,1005,4500.0,East,Clothing,2022-07-07,2023-03-10,2022,7,7
8,1003,7000.0,East,Electronics,2021-11-03,2023-02-28,2021,11,3
9,1006,11000.0,West,Furniture,2022-01-25,2023-03-15,2022,1,25
10,1001,4000.0,North,Clothing,2021-05-20,2023-04-01,2021,5,20


In [5]:
# 모듈 사용 후 코드
new_df = df1.copy()
new_df = dt_extract(new_df, 'signup_date', 'year', new_col='year') # df 인자로 new_df로 받아줘야 새로운컬럼들이 계속해서 추가됨, 원본 df만 인자로 받으면 new_df에는 원본 + 새 컬럼, 원본 + 새 컬럼 .. 이 반복될 뿐
new_df = dt_extract(new_df, 'signup_date', 'month', new_col='month')
new_df = dt_extract(new_df, 'signup_date', 'day', new_col='day')
new_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,customer_id,sales,region,product_category,signup_date,date,year,month,day
1,1001,5000.0,North,Electronics,2021-05-20,2023-01-15,2021,5,20
2,1002,12000.0,South,Clothing,2020-08-14,2023-02-10,2020,8,14
3,1003,NaN,East,Furniture,2021-11-03,2023-01-20,2021,11,3
4,1001,3000.0,North,Clothing,2021-05-20,2023-03-05,2021,5,20
5,1004,8000.0,West,Electronics,2022-01-25,2023-01-25,2022,1,25
6,1002,NaN,South,Furniture,2020-08-14,2023-02-20,2020,8,14
7,1005,4500.0,East,Clothing,2022-07-07,2023-03-10,2022,7,7
8,1003,7000.0,East,Electronics,2021-11-03,2023-02-28,2021,11,3
9,1006,11000.0,West,Furniture,2022-01-25,2023-03-15,2022,1,25
10,1001,4000.0,North,Clothing,2021-05-20,2023-04-01,2021,5,20


**> 함수 도입 효과**

- 이점 : 무슨무슨 컬럼이 새로 생성되었는지 보다 간단하게 확인가능, 데이터프레임을 새로 만들어서 번거롭게 기존 데이터셋에 join 하지 않아도 됨

- 아쉬운 점 : 기존 코드보다 길고(3줄 -> 4줄), 함수 반복적으로 사용됨. 함수는 year, month, day 중 한가지만 골라서 데이터 추출할 수 있게 되어있는 함수이기 때문. 추후 함수 모듈 개선 필요.


# Pandas_test 실전 종합 문제 
아래 작업을 순서대로 진행하는 함수 process_sales(df)를 만들어라.
- 결측치 처리 (sales는 평균으로)

- 'signup_date'를 datetime 변환하고 year 추출

- sales_level (High/Low) 추가 - 10000 기준

- 최종적으로 customer_id, sales, sales_level, year 컬럼만 남긴 DataFrame 반환

In [6]:
# 기존 코드
def sales_judge(df):
    if df.sales > 10000:
        return 'High'
    else:
        return 'Low'
        

def process_sales(df):
    df.sales = df.sales.fillna(df.sales.mean()) # 결측치 처리
    
    df['signup_date'] = pd.to_datetime(df['signup_date']) # datetime 변환, year 추출
    df['year'] = df['signup_date'].dt.year

    df['sales_level'] = df.apply(sales_judge, axis='columns')
    
    return df[['customer_id', 'sales', 'sales_level', 'year']]

process_sales(df1)

,customer_id,sales,sales_level,year
1,1001,5000.0,Low,2021
2,1002,12000.0,High,2020
3,1003,6812.5,Low,2021
4,1001,3000.0,Low,2021
5,1004,8000.0,Low,2022
6,1002,6812.5,Low,2020
7,1005,4500.0,Low,2022
8,1003,7000.0,Low,2021
9,1006,11000.0,High,2022
10,1001,4000.0,Low,2021


In [7]:
# 모듈 사용 후 코드
def sales_judge(df):
    if df.sales > 10000:
        return 'High'
    else:
        return 'Low'

def process_sales(df):
    df.sales = df.sales.fillna(df.sales.mean()) # 결측치 처리
    
    df = dt_extract(df, 'signup_date', 'year', new_col='year')

    df['sales_level'] = df.apply(sales_judge, axis='columns')
    
    return df[['customer_id', 'sales', 'sales_level', 'year']]

process_sales(df1)

,customer_id,sales,sales_level,year
1,1001,5000.0,Low,2021
2,1002,12000.0,High,2020
3,1003,6812.5,Low,2021
4,1001,3000.0,Low,2021
5,1004,8000.0,Low,2022
6,1002,6812.5,Low,2020
7,1005,4500.0,Low,2022
8,1003,7000.0,Low,2021
9,1006,11000.0,High,2022
10,1001,4000.0,Low,2021


**> 함수 도입 효과**

- 이점 : datetime 변환 및 year 데이터 추출 기능하는 코드 간략화 (2줄 -> 1줄), 시각적으로 이해하기 보다 간편

- 아쉬운 점 : - dt_extract의 인자들이 함수 기능에 있어 모두 필요한 인자들이긴 하나, 코드 자체가 너무 길어지기에 기존 코드보다 획기적으로 코드를 줄였다고 보기 어려움.

             - 데이터프레임을 인자로 받아야 해당 데이터프레임에 대한 접근이 가능한데 (ex. dt_extract(df, ... )), 멤버 접근 연산자(.)를 통해 데이터프레임으로부터 직접적으로 체이닝 연산을 할 수 있다면
             조금 더 코드를 간략화 할 수 있을 것 같음.

             - Monkey-Patching, Pandas Extention Accessor 등의 방법으로 해결이 가능하다고 함. 그러나 전자의 경우 전역으로 패치되어 충돌 위험과 유지보수 문제 등의 단점이 있어,
             네임스페이스 분리 및 Pandas 공식 방식을 따르는 후자의 방법이 권장된다고 함. 추후 함수 모듈 개선 필요.
             

# **group_agg**

In [8]:
from typing import Union, List, Dict # 꼭 임포트할것
def group_agg(
    df: pd.DataFrame,
    by: Union[str, List[str]],
    agg_dict: Dict[str, Union[str, List[str]]], # dictionary 키 밸류 값 구분에 :가 아니라 , 사용
    reset_index: bool = False
) -> pd.DataFrame:
    """
    그룹화 및 다중 프로퍼티에 대한 연산을 실행하는 함수.
    by: 그룹화할 컬럼 / agg_dict: key - 연산할 컬럼, value - 컬럼에 진행할 연산방식 / reset_index: True/False로 작동
    """
    result = df.groupby(by).agg(agg_dict)
    if reset_index:
        result = result.reset_index()
    return result

In [9]:
df2
# 예제 코드 모두 phase-1_hw 의 코드이므로 df2 사용

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932


# phase 1 hw - 전처리 4. 
카테고리별로 총 판매량, 총 수익, 평균 할인율을 비교하라.

In [10]:
# 기존 코드

# 방법 1
# total_sales = df2.groupby('Category').Sales.sum()
# discount_mean = df2.groupby('Category').Discount.mean()
# total_profit = df2.groupby('Category').Profit.sum()

# table = pd.DataFrame({'Total Sales': total_sales, 'Discount Mean': discount_mean, 'Total Profit': total_profit}, index=df2['Category'].unique())
# table

# 방법 2
df2.groupby('Category').agg({'Sales': 'sum', 'Discount': 'mean', 'Profit': 'sum'}).rename(columns={'Sales': 'Total Sales', 'Discount': 'Discount Mean', 'Profit': 'Total Profit'})

,Total Sales,Discount Mean,Total Profit
Category,,,
Furniture,741999.7953,0.173923,18451.2728
Office Supplies,719047.0320,0.157285,122490.8008
Technology,836154.0330,0.132323,145454.9481


In [11]:
# 모듈 사용 후 코드
group_agg(df2, 'Category', {'Sales': 'sum', 'Discount': 'mean', 'Profit': 'sum'}).rename(columns={'Sales': 'Total Sales', 'Discount': 'Discount Mean', 'Profit': 'Total Profit'})

,Total Sales,Discount Mean,Total Profit
Category,,,
Furniture,741999.7953,0.173923,18451.2728
Office Supplies,719047.0320,0.157285,122490.8008
Technology,836154.0330,0.132323,145454.9481


**> 함수 도입 효과**

- 이점 : groupby, agg를 통합하여 함수 하나로 기능하게 함으로써 가독성 향상. reset_index 같은 옵션이 늘어날 때마다 모든 호출 지점을 수정할 필요없이 함수 내부에서 한번에 처리 가능

- 아쉬운 점 : - 단순히 함수 내부에서 groupby, agg를 래핑한 것 뿐이기에 코드가 극적으로 간략화되지 않음. (실제로 예시 코드에서도 1줄 -> 1줄)

             - pandas에 익숙한 사용자는 groupby.agg를 통합한 기능을 하는 것을 바로 알 수 있지만, 초보 사용자는 내부 구조 파악을 위해 함수를 열어봐야할 수 있음.
              초보자가 해당 함수 내부 구조를 파악하는데 필요한 시간적 비용에 비해 개발 편의성을 효율적으로 끌어올릴 수 없음. 따라서 함수 도입의 의의가 부족하거나 없을 수도 있음.

             - 늘어난 옵션을 함수 내부에서 한번에 처리하더라도, 옵션을 켜고 끄는 것에 대한 코드 수정은 불가피함. 코드 수정을 하지 않기 위해 함수 내부에서 옵션을 일괄적으로 적용한다면,
               옵션이 필요하지 않은 케이스에도 일괄  적용 되므로 문제가 커짐. 따라서 옵션 관리의 이점이 크지 않을 수 있음.

# phase 1 hw - 전처리 5. 
각 고객이 몇 번이나 주문했는지와, 그들의 평균 구매 금액은 얼마인지 확인하라.

In [12]:
# 기존 코드
customer_sales_mean = df2.groupby('Customer ID').agg({'Order ID': 'count', 'Sales': 'mean'}) 
customer_sales_mean.rename(columns={'Order ID': 'Count', 'Sales': 'Sales Mean'})
# agg - 단일 프로퍼티의 다중 연산 : 리스트 사용 / 다중 프로퍼티 각각 연산 : 딕셔너리

,Count,Sales Mean
Customer ID,,
AA-10315,11,505.778182
AA-10375,15,70.426000
AA-10480,12,149.209333
AA-10645,18,282.607500
AB-10015,6,147.692667
...,...,...
XP-21865,28,84.809214
YC-21895,8,681.793750
YS-21880,12,560.037000


In [13]:
# 모듈 사용 후 코드
customer_sales_mean = group_agg(df2, 'Customer ID', {'Order ID': 'count', 'Sales': 'mean'}).rename(columns={'Order ID': 'Count', 'Sales': 'Sales Mean'})
customer_sales_mean

,Count,Sales Mean
Customer ID,,
AA-10315,11,505.778182
AA-10375,15,70.426000
AA-10480,12,149.209333
AA-10645,18,282.607500
AB-10015,6,147.692667
...,...,...
XP-21865,28,84.809214
YC-21895,8,681.793750
YS-21880,12,560.037000


**> 함수 도입 효과**

- 이점 : 위와 동일

- 아쉬운 점 : 위와 동일